In [1]:
from sklearn.neural_network import MLPClassifier
import pandas as pd 
import numpy as np

In [2]:
ci=pd.read_csv('/Users/EdvancerMac/Dropbox/March onwards/CBAP with R/Data/census_income.csv')

In [3]:
ci.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,Y
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
ci['Y']=np.where(ci['Y']==" <=50K",0,1)

In [5]:
ci.drop('education',axis=1,inplace=True)

In [6]:
cat_vars=ci.select_dtypes(['object']).columns

In [7]:
cat_vars

Index([u'workclass', u'marital.status', u'occupation', u'relationship',
       u'race', u'sex', u'native.country'],
      dtype='object')

In [8]:
for col in cat_vars:
    freqs=ci[col].value_counts()
    k=freqs.index[freqs>100][:-1]
    for cat in k:
        name=col+'_'+cat
        ci[name]=(ci[col]==cat).astype(int)
    del ci[col]
    print(col)

workclass
marital.status
occupation
relationship
race
sex
native.country


In [9]:
ci.shape

(32561, 48)

In [10]:
ci_train=ci.iloc[:28000,:]
ci_test=ci.iloc[28000:,:]

In [11]:
x_train=ci_train.drop('Y',axis=1)
y_train=ci_train['Y']
x_test=ci_test.drop('Y',axis=1)
y_test=ci_test['Y']

In [12]:
nn=MLPClassifier(hidden_layer_sizes=(20,10),activation='relu')

In [14]:
nn.fit(x_train,y_train)
predicted_test_score=nn.predict_proba(x_test)[:,1]

In [15]:
from sklearn.metrics import roc_auc_score

In [16]:
roc_auc_score(y_test,predicted_test_score)

0.65354278302047109

In [17]:
from sklearn.model_selection import KFold

In [18]:
hidden_layer_sizes= [(20,10),(20,11),(20,12),(20,13),(20,14)]

In [19]:
auc_list=[]

In [20]:
for h in hidden_layer_sizes:
    
    nn = MLPClassifier(hidden_layer_sizes=h,activation='relu')
    
    kf = KFold(n_splits=10)
    
    xval_err = 0
    
    for train, test in kf.split(x_train):
        
        nn.fit(x_train.loc[train], y_train[train])
        p = nn.predict_proba(x_train.loc[test])[:,1]
        xval_err += roc_auc_score(y_train[test],p)
    
    auc_10cv = xval_err/10
    print(h, ":",auc_10cv)

((20, 10), ':', 0.61718549284141155)
((20, 11), ':', 0.61611541962287286)
((20, 12), ':', 0.61356482101281729)
((20, 13), ':', 0.61267435749244403)
((20, 14), ':', 0.61767101762659005)
